# Randomised Iterative Improvement on TSP



Set classpath to the pre-compiled jar

In [7]:
@file:DependsOn("../build/libs/kglsm.jar")

Define imports

In [8]:
import com.sihvi.glsm.problem.TSP
import com.sihvi.glsm.sls.GLSMBuilder
import com.sihvi.glsm.sls.StateMachineTransition
import com.sihvi.glsm.space.PermutationSearchSpace
import com.sihvi.glsm.space.DiscreteSearchSpace
import com.sihvi.glsm.strategy.IIMode
import com.sihvi.glsm.strategy.IterativeImprovementStrategy
import com.sihvi.glsm.strategy.RandomWalkStrategy
import com.sihvi.glsm.transitionpredicate.NoImprovementPredicate
import com.sihvi.glsm.transitionpredicate.NotPredicate
import com.sihvi.glsm.transitionpredicate.ProbabilisticPredicate
import com.sihvi.glsm.memory.Memory
import com.sihvi.glsm.memory.BasicMemory
import com.sihvi.glsm.memory.BasicSolution

## Problem

We are using TSP problem as an example. Get an instance of the problem.

We use a280 instance from TSPLIB [

In [6]:
val dimensions = 2

val problemInstance = TSP.fromFile("a280.tsp")
val noPoints = problemInstance.size
println(problemInstance)

java.io.FileNotFoundException: a280.tsp (No such file or directory)
java.base/java.io.FileInputStream.open0(Native Method)
java.base/java.io.FileInputStream.open(FileInputStream.java:213)
java.base/java.io.FileInputStream.<init>(FileInputStream.java:155)
kotlin.io.FilesKt__FileReadWriteKt.forEachLine(FileReadWrite.kt:190)
kotlin.io.FilesKt__FileReadWriteKt.readLines(FileReadWrite.kt:219)
kotlin.io.FilesKt__FileReadWriteKt.readLines$default(FileReadWrite.kt:217)
com.sihvi.glsm.problem.TSP$Companion.fromFile(TSP.kt:65)
Line_5_jupyter.<init>(Line_5.jupyter.kts:3)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.

## Search space
Permutation search space defines two methods on an array of booleans: to get a neighbourhood and a random neighbour.

2-exchange neighbourhood is used for permutation search space

In [4]:
val space = PermutationSearchSpace(noPoints)

java.lang.NullPointerException
Line_3_jupyter.<init>(Line_3.jupyter.kts:1)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:96)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:41)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:51)
org.jetbrains.kotlin.jupyter.ReplForJupyt

## Memory
Basic memory that holds:
* Current solution and its cost
* Best solution and best cost
* Number of steps performed
* Number of steps performed without improvement (when it was expected)

In [ ]:
val initialSolution = space.getInitial()
val memory = BasicMemory(BasicSolution(initialSolution, problemInstance.evaluate(initialSolution)))
println(memory.bestSolution)

## GLSM and Strategies
Randomised Iterative Improvement consists of two strategies that are flipped probabilistically
* Iterative Best Improvement Strategy -- picks a solution from a neighbourhood that gives the best improvement
* Random Walk Strategy -- randomly picks a solution from a neighbourhood

The termination predicate of choice here is No Improvement Predicate, i.e. we terminate the search if there were n steps without improvement (in this case 10)

In [ ]:
val terminationPredicate = NoImprovementPredicate(10)

val wp = 0.1
val toRandomPredicate = ProbabilisticPredicate(to = wp)
val toIIPredicate = NotPredicate(toRandomPredicate)

val walk = RandomWalkStrategy<Int>()
val iterativeImprovement = IterativeImprovementStrategy<Int>(IIMode.BEST, true)

As all the components are defined, we can now build the GLSM with strategies and transitions between them 

In [ ]:
val glsm = GLSMBuilder<Int, BasicSolution<Int>, Memory<Int, BasicSolution<Int>>, DiscreteSearchSpace<Int>>()
        .addStrategy(iterativeImprovement)
        .addStrategy(walk)
        .addTransition(StateMachineTransition(0, 1, toRandomPredicate))
        .addTransition(StateMachineTransition(1, 0, toIIPredicate))
        .addTransition(StateMachineTransition(0, -1, terminationPredicate))
        .build()

# Solve

With everything ready we can run GLSM on our problem instance

In [ ]:
val finalSolution = glsm.solve(memory, space, problemInstance::evaluate)

println("Steps taken: " + memory.stepCount)
println("Solution: " + finalSolution.solution.joinToString(", "))
println("Cost: " + finalSolution.cost)

In [ ]:
memory.stepCount